In [13]:
# StackOverlow Developer Survey - Data Cleaning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

In [14]:
# 1. Load the dataset
print("1. Loading the dataset...")
df = pd.read_csv('../data/raw/survey_results_public.csv')
print("Dataset loaded successfully.")
print(f"Initial shape: {df.shape}")

1. Loading the dataset...
Dataset loaded successfully.
Initial shape: (49123, 170)


In [15]:
# 2. Basic Information
print("\n2. Basic information...")
print(f"Dataset size: {df.shape[0]} rows, {df.shape[1]} columns")


2. Basic information...
Dataset size: 49123 rows, 170 columns


In [16]:
# 3. Define target variable - We're predicting salary
print("\n3. Identifying target variable...")
target_column = 'ConvertedCompYearly'
if target_column in df.columns:
    print(f"Target variable: {target_column}")
    print(f"Target variable stats:\n{df[target_column].describe()}")
    
    # Check for missing values in target
    target_missing = df[target_column].isna().sum()
    print(f"Missing values in target: {target_missing} ({target_missing/len(df)*100:.2f}%)")
else:
    print(f"target variable '{target_column}' not found in dataset.")


3. Identifying target variable...
Target variable: ConvertedCompYearly
Target variable stats:
count    2.392800e+04
mean     1.017916e+05
std      4.619345e+05
min      1.000000e+00
25%      3.817100e+04
50%      7.538350e+04
75%      1.206302e+05
max      5.000000e+07
Name: ConvertedCompYearly, dtype: float64
Missing values in target: 25195 (51.29%)


In [17]:
# 4. Analyze missing values....")
print("\n4. Analyzing missing values...")
missing_counts = df.isnull().sum()
missing_percentage = (missing_counts / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing_counts.index,
    'Missing_Count': missing_counts.values,
    'Missing_Percentage': missing_percentage.values
}).sort_values('Missing_Percentage', ascending=False)

print("Top 10 columns with most missing data:")
print(missing_df.head(10))


4. Analyzing missing values...
Top 10 columns with most missing data:
                   Column  Missing_Count  Missing_Percentage
163       AIAgentObsWrite          48859           99.462574
96       SOTagsWant Entry          48693           99.124646
95        SOTagsHaveEntry          48666           99.069682
114     AIModelsWantEntry          48649           99.035075
161      AIAgentOrchWrite          48646           99.028968
54   JobSatPoints_15_TEXT          48459           98.648291
159      AIAgentKnowWrite          48358           98.442685
113     AIModelsHaveEntry          48348           98.422328
130    SO_Actions_15_TEXT          48300           98.324614
165       AIAgentExtWrite          48265           98.253364


In [18]:
# 5. Drop columns with excessive missing data
print("\n5. Dropping columns with excessive missing data...")
# Drop columns with more than 50% missing values
high_missing_cols = missing_df[missing_df['Missing_Percentage'] > 50]['Column'].tolist()
print(f"Dropping {len(high_missing_cols)} columns with > 50% missing data:")
for col in high_missing_cols:
    print(f" - {col} ({missing_df[missing_df['Column']==col]['Missing_Percentage'].values[0]:.1f}%)")
    # Print in alphabetical order
# if len(high_missing_cols) > 10:
#     print(f" ... and {len(high_missing_cols) - 10} more.")
    
df_cleaned = df.drop(columns=high_missing_cols)
print(f"Dataset shape after dropping high-missing columns: {df_cleaned.shape}")
# Print the difference in column size


5. Dropping columns with excessive missing data...
Dropping 71 columns with > 50% missing data:
 - AIAgentObsWrite (99.5%)
 - SOTagsWant Entry (99.1%)
 - SOTagsHaveEntry (99.1%)
 - AIModelsWantEntry (99.0%)
 - AIAgentOrchWrite (99.0%)
 - JobSatPoints_15_TEXT (98.6%)
 - AIAgentKnowWrite (98.4%)
 - AIModelsHaveEntry (98.4%)
 - SO_Actions_15_TEXT (98.3%)
 - AIAgentExtWrite (98.3%)
 - CommPlatformWantEntr (97.6%)
 - CommPlatformHaveEntr (97.0%)
 - DatabaseWantEntry (96.9%)
 - OfficeStackWantEntry (96.7%)
 - TechOppose_15_TEXT (96.7%)
 - TechEndorse_13_TEXT (95.9%)
 - DevEnvWantEntry (95.7%)
 - DatabaseHaveEntry (95.6%)
 - OfficeStackHaveEntry (94.7%)
 - WebframeWantEntry (94.7%)
 - AIAgentObserveSecure (94.5%)
 - DevEnvHaveEntry (94.4%)
 - PlatformWantEntry (93.7%)
 - LanguagesWantEntry (93.6%)
 - WebframeHaveEntry (93.3%)
 - AIAgentKnowledge (93.1%)
 - AIAgentOrchestration (92.3%)
 - AIAgentImpactStrongly disagree (92.3%)
 - PlatformHaveEntry (92.0%)
 - LanguagesHaveEntry (91.9%)
 - AIAg

In [19]:
# 6. Drop columns irrelevant to salary prediction
print("\n6. Dropping irrelevant columns...")
# These are columns that I deemed unlikely to help predict salary from exploratory analysis and domain knowledge
irrelevant_cols = [
    'ResponseId', # Unique identifier for each response
    'LearnCodeAI', # Did you spend time in the last year learning AI programming or AI-enabled tooling on your own or at work?
    # ^ AI not involved in salary prediction, I know some dumb programmers that learned AI
    'AILearnedHow', # How did you learn to code for AI in the past year? Select all that apply.
    # ^ Method in which AI was learned also not relevant
    'TechOppose_15_TEXT', # Text entry for tech oppose question
    'JobSatPoints_15_TEXT', # Text entry for job satisfaction question
    'Currency', # related to salary - target variable is already in USD
    'CompTotal', # related to salary - target variable is already in USD
    'LanguagesHaveEntry', # Lot of empty values, not relevant
    'LanguagesWantEntry', # Lot of empty values, not relevant
    'DatabaseAdmired', # May include useful info, will test with model later
    'WebframeAdmired', # May include useful info, will test with model later
    'DevEnvsHaveWorkedWith', # May include useful info, will test with model later
    'DevEnvsWantToWorkWith', # May include useful info, will test with model later
    'DevEnvsAdmired', # May include useful info, will test with model later
    'SOTagsHaveWorkedWith', # May include useful info, will test with model later
    'SOTagsWantToWorkWith', # May include useful info, will test with model later
    'SOTagsAdmired', # May include useful info, will test with model later
    'OfficeStackAsyncHaveWorkedWith', # To many different responses, responses are not related to each other too random
    'OfficeStackAsyncWantToWorkWith', # To many different responses, responses are not related to each other too random
    'OfficeStackAsyncAdmired', # To many different responses, responses are not related to each other too random
    'CommPlatformAdmired', # To many different responses, responses are not related to each other too random
    'AIModelsChoice', # Not relevant to salary prediction
    'AIModelsHaveWorkedWith', # Not relevant to salary prediction
    'AIModelsWantToWorkWith', # Not relevant to salary prediction
    'AIModelsAdmired', # Not relevant to salary prediction
    'AISelect', # Not relevant to salary prediction
    'AISent', # Not relevant to salary prediction
    'AIAcc', # Not relevant to salary prediction
    'AIComplex', # Not relevant to salary prediction
    'AIToolCurrently partially AI', # Not relevant to salary prediction
    'AIToolDon\'t plan to use AI for this task', # Not relevant to salary prediction
    'AIToolPlan to partially use AI', # Not relevant to salary prediction
    'AIToolPlan to mostly use AI', # Not relevant to salary prediction
    'AIToolCurrently mostly AI', # Not relevant to salary prediction
    'AIFrustration', # Not relevant to salary prediction
    'AIExplain', # Not relevant to salary prediction
    'AIAgents', # Not relevant to salary prediction
    'AIAgentChange', # Not relevant to salary prediction
    'AIAgent_Uses', # Not relevant to salary prediction
    'AgentUsesGeneral', # Not relevant to salary prediction
    'AIAgentImpactSomewhat agree', # Not relevant to salary prediction
    'AIAgentImpactNeutral', # Not relevant to salary prediction
    'AIAgentImpactSomewhat disagree', # Not relevant to salary prediction
    'AIAgentChallengesNeutral', # Not relevant to salary prediction
    'AIAgentChallengesSomewhat disagree', # Not relevant to salary prediction
    'AIAgentChallengesStrongly agree', # Not relevant to salary prediction
    'AIAgentChallengesStrongly disagree', # Not relevant to salary prediction
    'AIAgentExternal', # Not relevant to salary prediction
    'AIHuman', # Not relevant to salary prediction
    'AIOpen', # Not relevant to salary prediction
    # Add more irrelevant columns
]

# Filter to only keep columns that actually exist
irrelevant_cols = [col for col in irrelevant_cols if col in df_cleaned.columns]
df_cleaned = df_cleaned.drop(columns=irrelevant_cols)
print(f"Dropped {len(irrelevant_cols)} irrelevant columns")
print(f"Dataset shape after dropping irrelevant columns: {df_cleaned.shape}")


6. Dropping irrelevant columns...
Dropped 17 irrelevant columns
Dataset shape after dropping irrelevant columns: (49123, 82)


In [ ]:
# 7. Handle missing values in remaining columns
print("\n7. Handling missing values...")
numerical_cols = df_cleaned.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df_cleaned.select_dtypes(include=['object']).columns

# Impute numerical columns with median
for col in numerical_cols:
    if df_cleaned[col].isna().sum() > 0:
        df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].median())

# Impute categorical columns with 'Unknown'
for col in categorical_cols:
    if df_cleaned[col].isna().sum() > 0:
        df_cleaned[col] = df_cleaned[col].fillna('Unknown')
        


In [ ]:
# LanguageHaveWorkedWith, LanguageWantToWorkWith, LanguageAdmired are strings with multiple values separated by semicolons
# Need to convert these columns to be useful for the model
# Do the same with DatabaseHaveWorkedWith, DatabaseWantToWorkWith
# Do the same with PlatformHaveWorkedWith, PlatformWantToWorkWith
# Do the same with WebframeHaveWorkedWith, WebframeWantToWorkWith
# OpSysPersonal use, OpSysProfessional use
# CommPlatformHaveWorkedWith
# CommPlatformWantToWorkWith
